In [1]:
from importlib import reload
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import subprocess
from glob import glob
from pkg import *
from pkg.functions import convert2int, convert2str, get_params
from typing import List, Dict, Any
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from torchvision.transforms import v2
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
# datasets = [1, 2, 3, 4, 5, 6, 7, 8, 9]
datasets = [1, 2]

myPaths = path.PathManager(datasets=datasets)
myProcessor = process.Processor(paths=myPaths, datasets=datasets)
params = myProcessor.get_parameters()
print(params)
f.get_counts(paths=myPaths, datasets=datasets)

# myDatasetManager = data.DatasetManager(paths=myPaths, datasets=datasets)

# train_loader, test_loader = f.prepare(data_manager=myDatasetManager, batch_size=10)

Water backgrounds initialized: 2
{'01': {'clen': 0.15, 'photon_energy': 6000}, '02': {'clen': 0.15, 'photon_energy': 7000}}
Paths refreshed for dataset ['01', '02'].
Directory: peaks/01
	Total files: 45
	Normal images: 35
	Empty images: 10
Directory: labels/01
	Total files: 45
	Normal images: 35
	Empty images: 10
Directory: peaks_water_overlay/01
	Total files: 45
	Normal images: 35
	Empty images: 10
Directory: peaks/02
	Total files: 97
	Normal images: 75
	Empty images: 22
Directory: labels/02
	Total files: 97
	Normal images: 75
	Empty images: 22
Directory: peaks_water_overlay/02
	Total files: 97
	Normal images: 75
	Empty images: 22


In [3]:
f.check_attributes(paths=myPaths, datasets=f.convert2str(datasets), dir_type='peak')
print(myProcessor.get_parameters())

# f.retrieve_attributes(file_path=myPaths.total_paths.water_background[0])

c:\Users\ejsev\Desktop\cxls_hitfinder\images\peaks\01\empty_peak_01_00001.h5 conforms to expected attributes.
c:\Users\ejsev\Desktop\cxls_hitfinder\images\peaks\01\empty_peak_01_00001.h5:
Expected Peak: False, Actual Peak: False
c:\Users\ejsev\Desktop\cxls_hitfinder\images\peaks\01\empty_peak_01_00002.h5 conforms to expected attributes.
c:\Users\ejsev\Desktop\cxls_hitfinder\images\peaks\01\empty_peak_01_00002.h5:
Expected Peak: False, Actual Peak: False
c:\Users\ejsev\Desktop\cxls_hitfinder\images\peaks\01\empty_peak_01_00003.h5 conforms to expected attributes.
c:\Users\ejsev\Desktop\cxls_hitfinder\images\peaks\01\empty_peak_01_00003.h5:
Expected Peak: False, Actual Peak: False
c:\Users\ejsev\Desktop\cxls_hitfinder\images\peaks\01\empty_peak_01_00004.h5 conforms to expected attributes.
c:\Users\ejsev\Desktop\cxls_hitfinder\images\peaks\01\empty_peak_01_00004.h5:
Expected Peak: False, Actual Peak: False
c:\Users\ejsev\Desktop\cxls_hitfinder\images\peaks\01\empty_peak_01_00005.h5 conform

In [4]:
f.get_counts(paths=myPaths, datasets=datasets)
# %run process_directory.py ../../images  --percent_empty 0.5 

Paths refreshed for dataset ['01', '02'].
Directory: peaks/01
	Total files: 45
	Normal images: 35
	Empty images: 10
Directory: labels/01
	Total files: 45
	Normal images: 35
	Empty images: 10
Directory: peaks_water_overlay/01
	Total files: 45
	Normal images: 35
	Empty images: 10
Directory: peaks/02
	Total files: 97
	Normal images: 75
	Empty images: 22
Directory: labels/02
	Total files: 97
	Normal images: 75
	Empty images: 22
Directory: peaks_water_overlay/02
	Total files: 97
	Normal images: 75
	Empty images: 22


In [9]:
# transform = v2.Compose([
#     transform.TransformToTensor(),  # Custom transform to convert numpy array to tensor
#     v2.RandomRotation(degrees=15)         # Random rotation by +/- 15 degrees
# ])
transform = None
myDataManager = data.DatasetManager(paths=myPaths, datasets=datasets, transform=transform)
train_loader, test_loader = f.prepare(data_manager=myDataManager, batch_size=2)

Actual percentage of empty images: 22.535211267605636% across peaks, water_overlays, and labels directories.

Final dataset sizes - Peaks: 142, Labels: 142, Overlays: 142

Data prepared.
Train size: 113
Test size: 29
Batch size: 2
Number of batches in train_loader: 57 



In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
optimizer = optim.Adam
# optimizer = optim.SGD
scheduler = ReduceLROnPlateau

cfg = {
    "loader": [train_loader, test_loader],
    'batch_size': train_loader.batch_size,
    'optimizer': optimizer,
    'device': device,
    'scheduler': scheduler
    }


peak_config = eval.Peak_Detection_Configuration(myPaths, datasets, device, save_path='../models/peak_model.pt')
print(f'weights for peak : {peak_config.get_loss_weights()}')
photon_config = eval.Photon_Energy_Configuration(myPaths, datasets, device, save_path='../models/photon_model.pt')
print(f'weights for photon energy : {photon_config.get_loss_weights()}')
clen_config = eval.Camera_Length_Configureation(myPaths, datasets, device, save_path='../models/clen_model.pt')
print(f'weights for camera length : {clen_config.get_loss_weights()}')

diagram_save_path = '../../docs/diagrams/'

cuda


In [7]:
# train_loader, test_loader = f.prepare(data_manager=myDataManager, batch_size=1)

# for input, label, attribute in train_loader:
#     print(f'attributes : {attribute}')
#     peaks, _ = find_peaks(input[1].cpu().numpy().flatten(), height=25)
#     print(f'peaks : {peaks}')
#     print(f'length of peaks : {len(peaks)}')
#     plt.imshow(input[1].squeeze(0).squeeze(0).cpu().numpy())
#     plt.show()
#     break

In [8]:
a = train_eval.TrainTestModels(cfg, peak_config)
a.epoch_loop()
a.plot_loss_accuracy()
# a.find_optimal_threshold()
# a.plot_roc_curve()
a.plot_confusion_matrix()
a.save_model()

OutOfMemoryError: CUDA out of memory. Tried to allocate 34.11 GiB. GPU 0 has a total capacity of 6.00 GiB of which 4.95 GiB is free. Of the allocated memory 5.50 KiB is allocated by PyTorch, and 1.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
a.get_confusion_matrix()

array([[1., 0.],
       [1., 0.]])

In [ ]:
# b = train_eval.TrainTestModels(cfg, photon_config)
# b.epoch_loop()
# b.plot_loss_accuracy(diagram_save_path + 'photon_energy_loss_accuracy.png')
# b.plot_confusion_matrix(diagram_save_path + 'photon_energy_confusion_matrix.png')
# b.save_model()

In [ ]:
# photon_config.get_model_diagram('photon_energy_model.png', diagram_save_path, device)

In [ ]:
# b.get_confusion_matrix()

In [ ]:
# c = train_eval.TrainTestModels(cfg, clen_config)
# c.epoch_loop()
# c.plot_loss_accuracy(diagram_save_path + 'camera_length_loss_accuracy.png')
# c.plot_confusion_matrix(diagram_save_path + 'camera_length_confusion_matrix.png')
# c.save_model()

In [ ]:
# clen_config.get_model_diagram('camera_length_model.png', diagram_save_path, device)

In [ ]:
# c.get_confusion_matrix()

In [ ]:
# reload(pipe)
# torch.cuda.synchronize()
# torch.cuda.empty_cache()
# pipeline = pipe.ModelPipeline(peak_config, photon_config, clen_config, device)

In [ ]:
# train_loader, test_loader = f.prepare(data_manager=myDataManager, batch_size=1)

# for inputs, labels, attributes in train_loader:
#     inputs = inputs[1]
#     # print(f'input size : {inputs.size()}')
#     print(f'-- attributes: {attributes}')
#     results = pipeline.run(inputs.to(device))
#     print(f'-- results: {results}')
#     break